# Sentence 버전 #1 : 단어 시퀀스 

## 반복자
### 다음 항목을 반환하거나, 다음 항목이 없을 때 StopIteration 예외를 발생시키는, 인수를 받지 않는 \_\_next__() 메서드를 구현하는 객체.
### 파이썬 반복자는 \_\_iter__() 메서드도 구현하므로 반복형이기도 하다.

In [1]:
import re
import reprlib

RE_WORD = re.compile("\w+")

class Sentence :
    
    def __init__(self, text) :
        self.text = text
        self.words = RE_WORD.findall(text)
        
    # __iter__ 을 구현하지 않아도 __getitem__ 을 구현하면 인덱스가 0 부터 반복한다.
    def __getitem__(self, index) :
        return self.words[index]
    
    def __len__(self) :
        return len(self.words)
    
    def __repr__(self) :
        return f"Sentence({reprlib.repr(self.text)})"

In [4]:
s = Sentence('"The time has come", the Walrus said,')
s 

Sentence('"The time ha... Walrus said,')

In [5]:
for word in s :
    print(word)
    

The
time
has
come
the
Walrus
said


# Sentence 버전 #2 : 고전적인 반복자
## 반복형과 반복자의 차이
<br>

## 반복형 : 호출할 때마다 반복자를 새로 생성하는 \_\_iter__() 메서드를 가지고 있다.
## 반복자 : 개별 항목을 반환하는 \_\_next__() 메서드와 self를 반환하는 \_\_iter__() 메서드를 가지고 있다.

In [7]:
import re
import reprlib

RE_WORD = re.compile("\w+")

# 반복형
class Sentence :
    
    def __init__(self, text) :
        self.text = text
        self.words = RE_WORD.findall(text)
    
    def __repr__(self) :
        return f"Sentence({reprlib.repr(self.text)})"
    
    def __iter__(self) :
        return SentenceIterator(self.words)
    
# 반복자
class SentenceIterator :
    
    def __init__(self, words) :
        self.words = words
        self.index = 0
        
    def __next__(self) :
        try :
            word = self.words[self.index]
        except :
            raise StopIteration()
        self.index+=1
        return word
    
    def __iter__(self) :
        return self

In [8]:
s = Sentence('"The time has come", the Walrus said,')
s 

Sentence('"The time ha... Walrus said,')

# Sentence 버전 #3 : 제너레이터 함수

In [5]:
import re
import reprlib

RE_WORD = re.compile('\w+')

class Sentence :
    def __init__(self, text) :
        self.text = text
        self.words = RE_WORD.findall(text)
        
    def __repr__(self) :
        return f'Sentence({self.text})'
    
    def __iter__(self) :
        for word in self.words :
            yield word

In [12]:
sentence = Sentence("Lorem ipsum dolor sit amet.")
for word in sentence :
    print(word)

Lorem
ipsum
dolor
sit
amet


# Sentence 버전 #4 : 느긋한 구현

In [17]:
import re
import reprlib

RE_WORD = re.compile('\w+')

class Sentence :
    def __init__(self, text) :
        self.text = text
        
    def __repr__(self) :
        return f'Sentence({self.text})'
    
    # words 를 미리 만들지 않고 호출될 경우 하나씩 찾는다. -> 메모리 절약
    def __iter__(self) :
        for match in RE_WORD.finditer(self.text) :
            yield match.group()

In [16]:
sentence = Sentence("Lorem ipsum dolor sit amet.")
for word in sentence :
    print(word)

Lorem
ipsum
dolor
sit
amet


# Sentence 버전 #5 : 제너레이터 표현식

In [20]:
import re
import reprlib

RE_WORD = re.compile('\w+')

class Sentence :
    def __init__(self, text) :
        self.text = text
        
    def __repr__(self) :
        return f'Sentence({self.text})'
    
    # words 를 미리 만들지 않고 호출될 경우 하나씩 찾는다. -> 메모리 절약
    def __iter__(self) :
        return (match.group() for match in RE_WORD.finditer(self.text))

In [21]:
sentence = Sentence("Lorem ipsum dolor sit amet.")
for word in sentence :
    print(word)

Lorem
ipsum
dolor
sit
amet


# 예제 : 등차수열 제너레이터

In [40]:
class ArithmeticProgression :
    def __init__(self, begin, step, end=None) :
        self.begin = begin
        self.step = step
        self.end = end
        
    def __iter__(self) :
        # 신기하게 형변환하기..
        result = type(self.begin + self.step)(self.begin)
        forever = self.end is None
        index = 0
        while forever or result < self.end :
            yield result
            
            # 오차 누적을 줄이기 위해 result 에 self.step 을 더해나가지 않는다.
            index += 1
            result = self.begin + self.step * index

In [46]:
from decimal import Decimal
from fractions import Fraction


ap = ArithmeticProgression(0,1,3)
print(list(ap))

ap = ArithmeticProgression(0,0.5,3)
print(list(ap))

ap = ArithmeticProgression(0,1/3,1)
print(list(ap))

ap = ArithmeticProgression(0, Fraction(1,3), 1)
print(list(ap))

ap = ArithmeticProgression(0, Decimal('.1'), 0.3)
print(list(ap))

for x in ArithmeticProgression(0, Decimal('.1')) :
    if x > 1 : break
    print(x, end=' ')

[0, 1, 2]
[0.0, 0.5, 1.0, 1.5, 2.0, 2.5]
[0.0, 0.3333333333333333, 0.6666666666666666]
[Fraction(0, 1), Fraction(1, 3), Fraction(2, 3)]
[Decimal('0'), Decimal('0.1'), Decimal('0.2')]
0 0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1.0 